<a href="https://colab.research.google.com/github/hululuzhu/chinese-ai-writing-share/blob/main/training/2023_llm/2023_LLM_poem_ai_baichuan_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2023 LLM (baichuan-7b) 写诗 (WIP)
- 基础模型 https://huggingface.co/baichuan-inc/Baichuan-7B
- 数据来源 https://github.com/chinese-poetry/chinese-poetry
- 微调参数
  - 基础模型：int8 （7-8G GPU ram）
  - 微调方法：LoRA （16rank）

## 进程
- 12/25/2023：第一稿，走通流程
- 预计01/30/2024 发布LoRA到HuggingFace

# 准备阶段
- GPU （10G RAM以上，所以T4也可以）
- Transformer 需要特殊4.33.2版本，否则BaiChuan tokenizer报错，头痛


In [1]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-7785eea3-91b0-570f-9fdc-44fc31f42918)


In [2]:
!pip install -q nvidia-ml-py3
import nvidia_smi
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
nvidia_smi.nvmlShutdown()

print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

assert info.free > 1e10, (
    "Looks like your GPU is busy or not having enough 10G memory to continue")

  Preparing metadata (setup.py) ... done
Total memory: 42949672960
Free memory: 42479321088
Used memory: 470351872


In [3]:
!pip install -q bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q peft
# https://medium.com/@zhaozhiming/fine-tuning-large-language-models-with-llama-factory-34e288d3f381
!pip install transformers==4.33.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uni

In [4]:
# disable warnings unless needed
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# https://github.com/googlecolab/colabtools/issues/3409
import locale
# Required for handling chinese chars
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import json
import urllib.request
import pandas as pd
!pip install -q "tqdm>=4.36.1" > /tmp/na
from tqdm.notebook import tqdm
!pip install -q chinese-converter
import chinese_converter  # 繁体到简体需要
import pickle
import os
import pandas as pd
import numpy as np

In [ ]:
from datasets import Dataset, load_dataset
import numpy as np
import os
import pandas as pd
import pathlib
from peft import PeftModel, get_peft_config, get_peft_model, LoraConfig, TaskType, prepare_model_for_int8_training
import pickle
import sys
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, AutoModelForSeq2SeqLM, DataCollatorForLanguageModeling


# int8方式导入BaiChuan 7B模型
- 理论上可以int4，可以GPU内存减半

In [5]:
# again, the special version of Transfromer lib above is required, annoying
tokenizer = AutoTokenizer.from_pretrained(
    "baichuan-inc/Baichuan-7B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "baichuan-inc/Baichuan-7B", device_map="auto",
    load_in_8bit=True,
    trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

tokenization_baichuan.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan-7B:
- tokenization_baichuan.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan-7B:
- configuration_baichuan.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_baichuan.py:   0%|          | 0.00/28.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan-7B:
- modeling_baichuan.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/14.0G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [6]:
# 简单测试
inputs = tokenizer('登鹳雀楼->王之涣\n夜雨寄北->', return_tensors='pt')
inputs = inputs.to('cuda:0')
pred = model.generate(**inputs, max_new_tokens=64,repetition_penalty=1.1)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

登鹳雀楼->王之涣
夜雨寄北->李商隐
过华清宫绝句三首(其一)->杜牧
望岳>杜甫
春望>>杜甫
石壕吏-<<新唐书·元载传》


In [7]:
# 和之前Alpaca设置对应，但不一定正确
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"  # Allow batched inference

In [ ]:
# 验证阶段，之后换为False全部数据导入
IS_TEST_FLOW = True

In [13]:
# https://github.com/chinese-poetry/chinese-poetry, last update 04/18/2023
POEM_CONTENT = {
    'tang': {
        'total': 58,
        'pattern': "https://raw.githubusercontent.com/chinese-poetry/chinese-poetry/master/%E5%85%A8%E5%94%90%E8%AF%97/poet.tang.{0}.json"
    },
    'song': {
        'total': 255,
        'pattern': "https://raw.githubusercontent.com/chinese-poetry/chinese-poetry/master/%E5%85%A8%E5%94%90%E8%AF%97/poet.song.{0}.json"
    }
}

def get_poems(is_test=True, verbose=True):
  df_list = []
  for dynasty in POEM_CONTENT:
    size = 3 if is_test else POEM_CONTENT[dynasty]['total']
    pbar = tqdm(total=size, desc="Dynasty " + dynasty)
    for i in range(size):
      url = POEM_CONTENT[dynasty]['pattern'].format(i * 1000)
      if verbose:
        print(f"download {url} now")
      df_list.append(pd.read_json(url))
      pbar.update(1)
  return pd.concat(df_list)

In [16]:
df = get_poems(is_test=IS_TEST_FLOW, verbose=False)
df['concat_paragraphs'] = [''.join(map(str, l)) for l in df['paragraphs']]
df = df[['author', 'title', 'concat_paragraphs']]

def convert_schinese(tchinese):
  return chinese_converter.to_simplified(tchinese)

df['s_content'] = df.apply(lambda row: convert_schinese(''.join(row.concat_paragraphs)), axis=1)
df['s_title'] = df.apply(lambda row: convert_schinese(''.join(row.title)), axis=1)
df['s_author'] = df.apply(lambda row: convert_schinese(''.join(row.author)), axis=1)

my_df = df
print("my_df size", len(my_df))

Dynasty tang:   0%|          | 0/3 [00:00<?, ?it/s]

Dynasty song:   0%|          | 0/3 [00:00<?, ?it/s]

my_df size 6002


In [17]:
# 之前同repo的一些数据清理代码
MAX_AUTHOR_CHAR = 4
MAX_TITLE_CHAR = 12
MIN_CONTENT_CHAR = 20
MAX_CONTENT_CHAR = 32
BAD_TOKENS = " ()[]《》（）□{}abcdefgxyz一"

def trim_author_fn(row):
  return row.s_author[:MAX_AUTHOR_CHAR]

def trim_title_fn(row):
  trimed_title = row.s_title[:MAX_TITLE_CHAR]
  for b in BAD_TOKENS:
    trimed_title = trimed_title.replace(b, "")
  return trimed_title

def trim_content_fn(row):
  trimed_content = row.s_content[:MAX_CONTENT_CHAR]
  # # End with a period to avoid partial ending to confuse model
  for b in BAD_TOKENS:
    trimed_content = trimed_content.replace(b, "")
  last_period = trimed_content.rfind("。")
  return trimed_content[:last_period+1]
  # return trimed_content

# Trim the size, a soft copy to avoid the view/copy conflict warning
my_df['s_author_trim'] = my_df.copy().apply(trim_author_fn, axis=1)
my_df['s_title_trim'] = my_df.copy().apply(trim_title_fn, axis=1)
my_df['s_content_trim'] = my_df.copy().apply(trim_content_fn, axis=1)

print("my_df size", len(my_df))

my_df size 6002


In [19]:
# Title cannot be empty
empty_title_mask = (my_df['s_title_trim'].str.len() == 0)
too_short_cotent_mask = (my_df['s_content_trim'].str.len() <= MIN_CONTENT_CHAR)
invalid_mask = (('无正文' == my_df['s_content_trim']) | ('无正文' == my_df['s_author_trim']))
too_short_mask =  empty_title_mask | too_short_cotent_mask | invalid_mask
# filtered_my_df = my_df.loc[too_short_mask]
# filtered_my_df

my_df = my_df.loc[~too_short_mask][[
  's_author_trim', 's_title_trim', 's_content_trim']]
print("my_df size", len(my_df))

my_df size 5513


In [20]:
import re
result_dict = {
    's_author_trim': [],
    's_title_trim': [],
    's_content_trim': [],
}
for i, row in my_df.iterrows():
  c = row['s_content_trim']
  snippets = list(re.split('，|。|？', c))
  lens = [len(s) for s in snippets if s.strip() != '']
  if max(lens) != min(lens) or max(lens) not in [5, 7]:
    continue
  result_dict['s_author_trim'].append(row['s_author_trim'])
  result_dict['s_title_trim'].append(row['s_title_trim'])
  result_dict['s_content_trim'].append(c)
# print("get rid of ", sum(bad_items))
my_df = pd.DataFrame(data=result_dict)
print("left", len(my_df))

left 4173


In [21]:
my_df.sample(100)

,s_author_trim,s_title_trim,s_content_trim
2042,张九龄,感遇十二首十,我有异乡忆，宛在云溶溶。凭此目不觏，要之心所钟。
1099,李白,杂曲歌辞行路难三首三,有耳莫洗颍川水，有口莫食首阳蕨。含光混世贵无名，何用孤高比云月。
1496,张说,杂曲歌辞苏摩遮三,腊月凝阴积帝台，豪歌击鼓送寒来。油囊取得天河水，将添上寿万年杯。
3603,田锡,赠杨临察,昔日长安际会时，客情乡思两依依。金门次第俱升达，云路高低各奋飞。
196,则天皇后,唐享昊天乐第四,巍巍睿业广，赫赫圣基隆。菲德承先顾，祯符萃眇躬。
...,...,...,...
593,郭元振,相和歌辞王昭君三首三,闻有南河信，传闻杀画师。始知君惠重，更遣画蛾眉。
2107,释延寿,山居诗其二○,自甘疏拙懒经营，大道从来戒满盈。但起贪心迷有限，谁能触目悟无生。
3080,宋白,牡丹诗十首其七,风排香阵拂瑶墀，御苑新晴烂漫时。十五素娥羞水色，三千红面洗胭脂。
2717,李昉,赠邓洵美,忆昔词场共着鞭，当时鶑谷喜同迁。关河契阔三千里，音信稀疏二十年。


# 一直以来都是一首诗用两种方式（有/无作者）混合训练，是否最佳？

In [22]:
AUTHOR_PROMPT = "模仿："
TITLE_PROMPT = "作诗："
EOS_TOKEN = ''
def build_dataset_df(df, include_author=True):
  dfc = df.copy()
  if include_author:
    dfc['source_text'] = TITLE_PROMPT + df['s_title_trim'] + EOS_TOKEN + AUTHOR_PROMPT + df['s_author_trim']
  else:
    dfc['source_text'] = TITLE_PROMPT + df['s_title_trim']
  dfc['target_text'] = df['s_content_trim']
  dfc = dfc[['source_text', 'target_text']]
  return dfc


In [23]:
df_author_title_content = build_dataset_df(my_df, True)
df_author_title_content[100:105]

,source_text,target_text
100,作诗：守岁模仿：高宗皇帝,今宵冬律尽，来朝丽景新。花余凝地雪，条含煖吹分。
101,作诗：九月九日幸临渭亭登高得秋模仿：中宗皇帝,九月正乘秋，三杯兴已周。泛桂迎尊满，吹花向酒浮。
102,作诗：登骊山高顶寓目模仿：中宗皇帝,四郊秦汉国，八水帝王都。阊阖雄里闬，城阙壮规模。
103,作诗：幸秦始皇陵模仿：中宗皇帝,眷言君失德，骊邑想秦余。政烦方改篆，愚俗乃焚书。
104,作诗：立春日游苑迎春模仿：中宗皇帝,神皐福地三秦邑，玉台金阙九仙家。寒光犹恋甘泉树，淑景偏临建始花。


In [24]:
df_title_content = build_dataset_df(my_df, False)
df_title_content[100:105]

,source_text,target_text
100,作诗：守岁,今宵冬律尽，来朝丽景新。花余凝地雪，条含煖吹分。
101,作诗：九月九日幸临渭亭登高得秋,九月正乘秋，三杯兴已周。泛桂迎尊满，吹花向酒浮。
102,作诗：登骊山高顶寓目,四郊秦汉国，八水帝王都。阊阖雄里闬，城阙壮规模。
103,作诗：幸秦始皇陵,眷言君失德，骊邑想秦余。政烦方改篆，愚俗乃焚书。
104,作诗：立春日游苑迎春,神皐福地三秦邑，玉台金阙九仙家。寒光犹恋甘泉树，淑景偏临建始花。


# 转化为Trainer喜欢的数据格式

In [30]:
# Copied from Alpaca-LoRA, notice input_ids, attention_mask, and labels are
# default expected columns in huggingface dataset lib
def tokenize(tokenizer, prompt, cutoff_len, add_eos_token=True):
  # there's probably a way to do this with the tokenizer settings
  # but again, gotta move fast
  result = tokenizer(
      prompt,
      truncation=True,
      max_length=cutoff_len,
      padding=False,
      return_tensors=None,
  )
  if (
      result["input_ids"][-1] != tokenizer.eos_token_id
      and len(result["input_ids"]) < cutoff_len
      and add_eos_token
  ):
    result["input_ids"].append(tokenizer.eos_token_id)
    result["attention_mask"].append(1)

  # result["labels"] = copy.deepcopy(result["input_ids"])
  result["labels"] = result["input_ids"].copy()
  return result


# Max num of tokens (including prompt and output), chinese encoding takes more
# than # of chars as observed
CUTOFF_LEN = 96
# Predict training prompt as well to increase quality as Alpaca Lora does.
# Turn off to speedup, but might affect quality.
TRAIN_ON_INPUT = False

# Branched from Alpaca-LoRA
def tokenize_fn(data_point):
  prompt_in, prompt_out = data_point['source_text'], data_point['target_text']
  full_prompt = prompt_in + prompt_out
  tokenized_full_prompt = tokenize(tokenizer, full_prompt, CUTOFF_LEN)
  if not TRAIN_ON_INPUT:
    user_prompt = prompt_in
    tokenized_user_prompt = tokenize(tokenizer, user_prompt, CUTOFF_LEN, add_eos_token=False)
    user_prompt_len = len(tokenized_user_prompt["input_ids"])
    tokenized_full_prompt["labels"] = [
        -100 # special id for skipping
    ] * user_prompt_len + tokenized_full_prompt["labels"][user_prompt_len:]
  return tokenized_full_prompt


train_ds = Dataset.from_pandas(df_title_content)
train_ds = train_ds.flatten()
tokenized_train_ds = train_ds.map(
    tokenize_fn,
    remove_columns=['source_text', 'target_text'],
)

Map:   0%|          | 0/4173 [00:00<?, ? examples/s]

In [31]:
# Optionally check a few examples by decoding the inputs
for i in range(100, 103):
  print("token length", len(tokenized_train_ds['input_ids'][i]))
  print(tokenizer.decode(tokenized_train_ds['input_ids'][i]))
  print("Label ids", tokenized_train_ds['labels'][i])
  print()

token length 31
 作诗：守岁今宵冬律尽，来朝丽景新。花余凝地雪，条含煖吹分。</s>
Label ids [-100, -100, -100, -100, -100, -100, 31541, 33783, 32324, 31859, 31884, 72, 31192, 32283, 32126, 31768, 31226, 73, 31528, 32012, 32837, 31218, 32269, 72, 31594, 31939, 41715, 32886, 31221, 73, 2]

token length 41
 作诗：九月九日幸临渭亭登高得秋九月正乘秋，三杯兴已周。泛桂迎尊满，吹花向酒浮。</s>
Label ids [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 32110, 31245, 31383, 32329, 32208, 72, 31318, 32576, 31906, 31453, 31599, 73, 32830, 32780, 32068, 32518, 31672, 72, 32886, 31528, 31490, 31805, 32845, 73, 2]

token length 35
 作诗：登骊山高顶寓目四郊秦汉国，八水帝王都。阊阖雄里闬，城阙壮规模。</s>
Label ids [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 31533, 33826, 32884, 31976, 31200, 72, 31978, 31323, 32577, 31702, 31228, 73, 38289, 35402, 32409, 31304, 47737, 72, 31454, 35270, 32849, 7517, 73, 2]



# 设置LoRA

In [32]:
original_8bit_llama_model = model

In [35]:
model = prepare_model_for_int8_training(original_8bit_llama_model)

config = LoraConfig(
    r=16,
    lora_alpha=32, # scaling param related to r, reuse alpaca-lora
    # only W-pack works https://github.com/beyondguo/LLM-Tuning/blob/master/baichuan_lora_tuning.py
    target_modules=['W_pack'],  # ["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

lora_model = get_peft_model(original_8bit_llama_model, config)

In [44]:
# Check out meaning of the Chinese char using ChatGPT
def eval_model(my_model, examples=["标题： 作诗：秋思\n诗歌： ",
                                   "标题： 作诗：佳人有约\n诗歌： ",
                                   "标题： 作诗：冬雪 模仿：杜甫\n诗歌： ",
                                   "标题： 作诗：冬雪 模仿：苏轼\n诗歌： "]):
  for p_in in examples:
    inputs = tokenizer(p_in, return_tensors='pt')
    inputs = inputs.to('cuda:0')
    pred = my_model.generate(**inputs, max_new_tokens=64,repetition_penalty=1.1)
    print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

In [38]:
eval_model(lora_model)

标题： 作诗：秋思
诗歌： 《秋思》作者:李文涛(一)落叶飘零，寒风萧瑟。天高云淡，雁去无声。菊花绽放，枫红似火。月光如水，思念成河。 (二)夜深人静，孤灯独坐。举
标题： 佳人有约：秋思
诗歌： 《秋思》作者:李建国(一)落叶飘零，寒风萧瑟。天高云淡，雁去无声。菊花绽放，枫红似火。月光如水，夜色朦胧。思念故乡，情深意浓。 (二)窗


## 这里所有参数都要重新考虑， 特别以下
- per_device_train_batch_size
- gradient_accumulation_steps
- learning_rate

In [39]:
# As you can tell, I even omitted eval_dataset for this demo :(
trainer = transformers.Trainer(
    model=lora_model,
    train_dataset=tokenized_train_ds,
    args=transformers.TrainingArguments(
        # increased batch size will significantly increase GPU requirement here
        # Decrease to 4 if you have less than 16G vram
        # Batch = 4, probably 8.3-8.8G vram
        # Batch = 16, 9.5G+
        # Batch = 32, 11G+
        # Batch = 64, 14G+
        per_device_train_batch_size=32,
        gradient_accumulation_steps=2,
        warmup_steps=8,
        num_train_epochs=2,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=20,
        output_dir='outputs',
        remove_unused_columns=False,
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True,
    ),
)
lora_model.config

BaiChuanConfig {
  "_name_or_path": "baichuan-inc/Baichuan-7B",
  "architectures": [
    "BaiChuanForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "baichuan-inc/Baichuan-7B--configuration_baichuan.BaiChuanConfig",
    "AutoModelForCausalLM": "baichuan-inc/Baichuan-7B--modeling_baichuan.BaiChuanForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "baichuan",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true,
    "quant_method": "bitsandbytes"
  },
  "rms_no

# 开炼！

In [40]:
trainer.train()

Step,Training Loss
20,3.989500
40,3.684300
60,3.572700
80,3.547200
100,3.510200
120,3.538400


TrainOutput(global_step=130, training_loss=3.6314940525935246, metrics={'train_runtime': 514.7163, 'train_samples_per_second': 16.215, 'train_steps_per_second': 0.253, 'total_flos': 1.6803104830390272e+16, 'train_loss': 3.6314940525935246, 'epoch': 1.98})

## 测试炼后结果

In [45]:
eval_model(lora_model)

标题： 作诗：秋思
诗歌： 寒蝉凄切，对长亭晚，骤雨初歇。都门帐饮无绪，留恋处、兰舟催发。执手相看泪眼，竟无语凝噎。念去去千里烟波，暮霭沉沉楚天阔。
标题： 作诗：佳人有约
诗歌： 红妆翠袖，玉貌花容。风流自是天上种，不比人间富贵荣华。
标题： 作诗：冬雪 模仿：杜甫
诗歌： 严陵滩上白蘋风，江北江南雪正浓。
标题： 作诗：冬雪 模仿：苏轼
诗歌： 梅花已报春消息，红蕊繁枝满院香。


# TODO：上传LoRA到HF